## Searching for bouts for a day of ephys recording
- microphone wav file is first exported in sglx_pipe-dev-sort-bouts-s_b1253_21-20210614
- bouts are extracted in searchbout_s_b1253_21-ephys

In [1]:
import os
import glob
import socket
import logging
import pickle
import numpy as np
import pandas as pd
from scipy.io import wavfile
from scipy import signal

### Fuck matplotlib, I'm using poltly now
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from importlib import reload

logger = logging.getLogger()
handler = logging.StreamHandler()
formatter = logging.Formatter(
        '%(asctime)s %(name)-12s %(levelname)-8s %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.INFO)

logger.info('Running on {}'.format(socket.gethostname()))

2021-08-28 00:02:25,421 root         INFO     Running on pakhi


In [2]:
from ceciestunepipe.file import bcistructure as et
from ceciestunepipe.util.sound import boutsearch as bs

### Get the file locations for a session (day) of recordings

In [15]:
reload(et)
sess_par = {'bird': 's_b1253_21',
           'sess': '2021-07-18',
           'sort': 2}


exp_struct = et.get_exp_struct(sess_par['bird'], sess_par['sess'], ephys_software='sglx')

raw_folder = exp_struct['folders']['sglx']

derived_folder = exp_struct['folders']['derived']

bouts_folder = os.path.join(derived_folder, 'bouts_ceciestunepipe')

sess_bouts_file = os.path.join(bouts_folder, 'bout_sess_auto.pickle')
sess_bouts_curated_file = os.path.join(bouts_folder, 'bout_curated.pickle')

os.makedirs(bouts_folder, exist_ok=True)

In [6]:
exp_struct['folders']

{'bird': '/mnt/sphere/speech_bci/raw_data/s_b1253_21',
 'raw': '/mnt/sphere/speech_bci/raw_data/s_b1253_21/2021-07-18',
 'sglx': '/mnt/sphere/speech_bci/raw_data/s_b1253_21/2021-07-18/sglx',
 'kwik': '/scratch/earneodo/s_b1253_21/sglx/kwik/2021-07-18',
 'processed': '/mnt/sphere/speech_bci/processed_data/s_b1253_21/2021-07-18/sglx',
 'derived': '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-07-18/sglx',
 'tmp': '/scratch/earneodo/tmp',
 'msort': '/scratch/earneodo/s_b1253_21/sglx/msort/2021-07-18',
 'ksort': '/scratch/earneodo/s_b1253_21/sglx/ksort/2021-07-18'}

### load concatenated the files of the session

In [9]:
def read_session_auto_bouts(exp_struct):
    # list all files of the session
    # read into list of pandas dataframes and concatenate
    # read the search parameters of the first session
    # return the big pd and the search params
    derived_folder = exp_struct['folders']['derived']
    bout_pd_files = et.get_sgl_files_epochs(derived_folder, file_filter='bout_auto.pickle')
    search_params_files = et.get_sgl_files_epochs(derived_folder, file_filter='bout_search_params.pickle')
    print(bout_pd_files)
    hparams=None
    with open(search_params_files[0], 'rb') as fh:
        hparams = pickle.load(fh)
    
    bout_pd = pd.concat([pd.read_pickle(p) for p in bout_pd_files[:]])
    
    return bout_pd, hparams

bout_pd, hparams = read_session_auto_bouts(exp_struct)

['/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-07-18/sglx/0610_g0/bout_auto.pickle', '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-07-18/sglx/1615_g0/bout_auto.pickle']


In [10]:
bout_pd['file'].values

array(['/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-07-18/sglx/0610_g0/wav_mic.wav',
       '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-07-18/sglx/0610_g0/wav_mic.wav',
       '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-07-18/sglx/0610_g0/wav_mic.wav',
       '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-07-18/sglx/0610_g0/wav_mic.wav',
       '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-07-18/sglx/0610_g0/wav_mic.wav',
       '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-07-18/sglx/0610_g0/wav_mic.wav',
       '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-07-18/sglx/0610_g0/wav_mic.wav',
       '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-07-18/sglx/0610_g0/wav_mic.wav',
       '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-07-18/sglx/0610_g0/wav_mic.wav',
       '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-07-18/sglx/0610_g0/wav_mic.wav',
       '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-07-18/sglx

###### if it wasnt saved (which is a bad mistake), read the sampling rate from the first file in the session

In [11]:
def sample_rate_from_wav(wav_path):
    x, sample_rate = wavfile.read(wav_path)
    return sample_rate

if hparams['sample_rate'] is None:
    one_wav_path = bpd.loc[0, 'file']
    logger.info('Sample rate not saved in parameters dict, searching it in ' + one_wav_path)
    hparams['sample_rate'] = sample_rate_from_wav(one_wav_path)

In [12]:
def cleanup(bout_pd: pd.DataFrame):
    ## check for empty waveforms (how woudld THAT happen???)
    bout_pd['valid_waveform'] = bout_pd['waveform'].apply(lambda x: (False if x.size==0 else True))
    
    # valid is & of all the validated criteria
    bout_pd['valid'] = bout_pd['valid_waveform']
    
    ## fill in the epoch
    bout_pd['epoch'] = bout_pd['file'].apply(lambda x: et.split_path(x)[-2])
    
    # drop not valid and reset index
    bout_pd.drop(bout_pd[bout_pd['valid']==False].index, inplace=True)
    bout_pd.reset_index(drop=True, inplace=True)
    
    # set all to 'confusing' (unchecked) and 'bout_check' false (not a bout)
    bout_pd['confusing'] = True
    bout_pd['bout_check'] = False

cleanup(bout_pd)

2021-08-28 00:04:37,654 numexpr.utils INFO     Note: NumExpr detected 32 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2021-08-28 00:04:37,655 numexpr.utils INFO     NumExpr defaulting to 8 threads.


In [13]:
bout_pd

,start_ms,end_ms,start_sample,end_sample,p_step,rms_p,peak_p,bout_check,file,len_ms,syl_in,n_syl,peaks_p,n_peaks,l_p_ratio,waveform,valid_waveform,valid,epoch,confusing
0,4539970,4545905,113499250,113647625,"[21.011606223298124, 42.12113172158324, 10.393...",25.943562,341.681207,False,/mnt/sphere/speech_bci/derived_data/s_b1253_21...,5935,"[[4539970, 4540015], [4540150, 4540185], [4540...",12,"[6, 15, 37, 45, 76, 96, 195, 219, 232, 249, 26...",42,141.309524,"[-39, -29, -36, -39, -39, -49, -16, -26, -29, ...",True,True,0610_g0,True
1,4746765,4758305,118669125,118957625,"[45.74815033265438, 82.1746515082166, 44.94570...",25.943562,286.314774,False,/mnt/sphere/speech_bci/derived_data/s_b1253_21...,11540,"[[4746765, 4746805], [4746815, 4746880], [4746...",28,"[1, 16, 28, 35, 48, 87, 175, 215, 254, 315, 33...",67,172.238806,"[-79, -67, -72, -71, -53, -57, -59, -55, -68, ...",True,True,0610_g0,True
2,5198415,5205645,129960375,130141125,"[15.595449730810868, 27.78407209544905, 44.294...",25.943562,325.224862,False,/mnt/sphere/speech_bci/derived_data/s_b1253_21...,7230,"[[5198415, 5198595], [5199855, 5199885], [5200...",10,"[7, 21, 29, 140, 149, 279, 291, 311, 321, 358,...",52,139.038462,"[-7, 4, 10, -19, 11, -11, -6, -18, -11, -4, -2...",True,True,0610_g0,True
3,5450700,5457220,136267500,136430500,"[90.74580028415619, 142.25687342792727, 58.280...",25.943562,404.323039,False,/mnt/sphere/speech_bci/derived_data/s_b1253_21...,6520,"[[5450700, 5450880], [5450890, 5450965], [5451...",16,"[1, 8, 19, 29, 42, 49, 59, 92, 100, 142, 152, ...",63,103.492063,"[19, 12, 18, 26, 28, 20, 28, 35, 32, 15, 32, 3...",True,True,0610_g0,True
4,5490110,5497160,137252750,137429000,"[54.6112447277905, 61.93353795083646, 52.55419...",25.943562,291.575781,False,/mnt/sphere/speech_bci/derived_data/s_b1253_21...,7050,"[[5490110, 5490210], [5490325, 5490360], [5490...",19,"[1, 10, 24, 33, 44, 53, 68, 109, 120, 147, 154...",52,135.576923,"[29, 62, 36, 37, 59, 41, 73, 48, 73, 73, 62, 6...",True,True,0610_g0,True
5,5659290,5666155,141482250,141653875,"[37.40769419124567, 53.71189699642886, 61.7635...",25.943562,433.694514,False,/mnt/sphere/speech_bci/derived_data/s_b1253_21...,6865,"[[5659290, 5659320], [5660390, 5660535], [5660...",9,"[2, 211, 221, 228, 236, 243, 262, 273, 310, 32...",42,163.452381,"[-4, 5, 5, 21, -6, -1, -3, -5, 19, 4, 1, 15, 1...",True,True,0610_g0,True
6,6300475,6351245,157511875,158781125,"[6.706255109451581, 10.075744685561203, 16.423...",25.943562,552.128597,False,/mnt/sphere/speech_bci/derived_data/s_b1253_21...,50770,"[[6300475, 6300800], [6302005, 6302070], [6302...",139,"[7, 24, 35, 45, 52, 59, 287, 307, 318, 328, 33...",806,62.990074,"[-39, -39, -62, -39, -60, -54, -55, -47, -60, ...",True,True,0610_g0,True
7,6505945,6517800,162648625,162945000,"[8.244111607067131, 10.893227473027403, 7.3424...",25.943562,349.234982,False,/mnt/sphere/speech_bci/derived_data/s_b1253_21...,11855,"[[6505945, 6505975], [6506570, 6506600], [6507...",21,"[4, 37, 67, 126, 143, 168, 183, 264, 274, 307,...",78,151.987179,"[-18, -16, -39, -11, -13, -6, -32, -13, -15, 1...",True,True,0610_g0,True
8,7102500,7140935,177562500,178523375,"[6.728729995305173, 19.523343475805618, 15.045...",25.943562,565.797124,False,/mnt/sphere/speech_bci/derived_data/s_b1253_21...,38435,"[[7102500, 7102600], [7103175, 7104375], [7105...",105,"[1, 12, 138, 146, 155, 164, 175, 183, 191, 199...",563,68.268206,"[33, 25, 14, 30, 37, 16, 25, 34, 28, 45, 43, 3...",True,True,0610_g0,True
9,7180615,7212520,179515375,180313000,"[20.33999544347377, 30.11012692971911, 30.4198...",25.943562,562.570087,False,/mnt/sphere/speech_bci/derived_data/s_b1253_21...,31905,"[[7180615, 7180725], [7180785, 7181365], [7182...",102,"[15, 34, 49, 57, 70, 78, 86, 93, 103, 112, 120...",486,65.648148,"[-25, -27, -28, -22, -54, -26, -19, -22, -22, ...",True,True,0610_g0,True


In [57]:
reload(et)

<module 'ceciestunepipe.file.bcistructure' from '/mnt/cube/earneodo/repos/ceciestunepipe/ceciestunepipe/file/bcistructure.py'>

#### compute the spectrograms

In [14]:
bout_pd['spectrogram'] = bout_pd['waveform'].apply(lambda x: bs.gimmepower(x, hparams)[2])
logger.info('saving bout pandas with spectrogram to ' + sess_bouts_file)
bout_pd.to_pickle(sess_bouts_file)

2021-08-28 00:05:31,806 root         INFO     saving bout pandas with spectrogram to /mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-07-18/sglx/bouts_ceciestunepipe/bout_auto.pickle


In [ ]:
bout_pd.head(2)

In [ ]:
bout_pd['file'][0]

## inspect the bouts and curate them

#### visualize one bout

In [94]:
bout_pd.iloc[0]

start_ms                                                    4124605
end_ms                                                      4132815
start_sample                                              103115125
end_sample                                                103320375
p_step            [20.019876899602302, 47.830789003771706, 37.87...
rms_p                                                     25.943562
peak_p                                                   476.650473
bout_check                                                    False
file              /mnt/sphere/speech_bci/derived_data/s_b1253_21...
len_ms                                                         8210
syl_in            [[4124605, 4124705], [4124765, 4124800], [4124...
n_syl                                                            19
peaks_p           [4, 12, 20, 33, 42, 53, 62, 72, 80, 101, 119, ...
n_peaks                                                          71
l_p_ratio                                       

In [17]:
import plotly.express as px
import plotly.graph_objects as go

from ipywidgets import widgets

In [18]:
def viz_one_bout(df: pd.Series, sub_sample=1):
    # get the power and the spectrogram
    sxx = df['spectrogram'][:, ::sub_sample]
    x = df['waveform'][::sub_sample]
    
    # the trace
    tr_waveform = go.Scatter(y=x)
    figwidg_waveform = go.FigureWidget(data=[tr_waveform],
                                      layout= {'height': 300,'width':1000})

    # the spectrogram
    fig_spectrogram = px.imshow(sxx, 
                                     labels={}, 
                                     color_continuous_scale='Inferno',
                                    aspect='auto')

    fig_spectrogram.update_layout(width=1000, height=300, coloraxis_showscale=False)
    fig_spectrogram.update_xaxes(showticklabels=False)
    fig_spectrogram.update_yaxes(showticklabels=False)
    
    
    figwidg_spectrogram = go.FigureWidget(fig_spectrogram)
    
    display(widgets.VBox([figwidg_waveform,
              figwidg_spectrogram]))
    

viz_one_bout(bout_pd.iloc[24])

    'data': [{'type': 'scatter',
              'uid': 'c4b31302-c370-4e74-9559-f…

In [19]:
bout_pd.head(2)

,start_ms,end_ms,start_sample,end_sample,p_step,rms_p,peak_p,bout_check,file,len_ms,...,n_syl,peaks_p,n_peaks,l_p_ratio,waveform,valid_waveform,valid,epoch,confusing,spectrogram
0,4539970,4545905,113499250,113647625,"[21.011606223298124, 42.12113172158324, 10.393...",25.943562,341.681207,False,/mnt/sphere/speech_bci/derived_data/s_b1253_21...,5935,...,12,"[6, 15, 37, 45, 76, 96, 195, 219, 232, 249, 26...",42,141.309524,"[-39, -29, -36, -39, -39, -49, -16, -26, -29, ...",True,True,0610_g0,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,4746765,4758305,118669125,118957625,"[45.74815033265438, 82.1746515082166, 44.94570...",25.943562,286.314774,False,/mnt/sphere/speech_bci/derived_data/s_b1253_21...,11540,...,28,"[1, 16, 28, 35, 48, 87, 175, 215, 254, 315, 33...",67,172.238806,"[-79, -67, -72, -71, -53, -57, -59, -55, -68, ...",True,True,0610_g0,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


## use it in a widget


#### add a 'confusing' label, for not/sure/mixed.
we want to avoid having things we are not sure of in the training dataset

In [20]:
bout_pd.reset_index(drop=True, inplace=True)

In [21]:
## Set confusing by default, will only be False once asserted bout/or not
bout_pd['confusing'] = True
bout_pd['bout_check'] = False

In [22]:
### Create a counter object (count goes 1:1 to DataFrame index)
from traitlets import CInt, link

class Counter(widgets.DOMWidget):
    value = CInt(0)
    value.tag(sync=True)

In [23]:
class VizBout():
    def __init__(self, hparams, bouts_pd):
        self.bout = None
        self.bouts_pd = bouts_pd
        self.bout_series = None
        self.is_bout = None
        self.is_confusing = None
        
        self.bout_counter = None
        self.bout_id = None
        
        self.buttons = {}
        self.m_pick = None
        
        
        self.fig_waveform = None
        self.fig_spectrogram = None
        
        self.figwidg_waveform = None
        self.figwidg_spectrogram = None
        
        self.fig_width = 2
        self.sub_sample = 10
        
        self.x = None
        self.sxx = None
        self.tr_waveform = None
        
        self.s_f = hparams['sample_rate']
        
        self.init_fig()
        self.init_widget()
        self.show()
        
    def init_fig(self):
        # the trace
        self.tr_waveform = go.Scatter(y=np.zeros(500))
        self.figwidg_waveform = go.FigureWidget(data=[self.tr_waveform],
                                               layout={'width': 1000, 'height':300})
        
        # the spectrogram
        self.fig_spectrogram = px.imshow(np.random.rand(500, 500), 
                                         labels={}, 
                                         color_continuous_scale='Inferno',
                                        aspect='auto')
        
        self.fig_spectrogram.update_layout(width=1000, height=300, coloraxis_showscale=False)
        self.fig_spectrogram.update_xaxes(showticklabels=False)
        self.fig_spectrogram.update_yaxes(showticklabels=False)
        self.figwidg_spectrogram = go.FigureWidget(self.fig_spectrogram)
        
        
    def init_widget(self):
        # declare elements
        # lay them out
        #
        
        self.bout_counter = Counter()
        self.is_bout = widgets.Checkbox(description='is bout')
        self.is_confusing = widgets.Checkbox(description='Not sure or mixed')
        
        self.buttons['Next'] = widgets.Button(description="Next", button_style='info',
                                              icon='plus')   
        self.buttons['Prev'] = widgets.Button(description="Prev", button_style='warning',
                                              icon='minus')
        self.buttons['Check'] = widgets.Button(description="Check", button_style='success', 
                                               icon='check')
        self.buttons['Uncheck'] = widgets.Button(description="Uncheck", button_style='danger',
                                                 icon='wrong')
        
        [b.on_click(self.button_click) for b in self.buttons.values()]
        
        left_box = widgets.VBox([self.buttons['Prev'], self.buttons['Uncheck']])
        right_box = widgets.VBox([self.buttons['Next'], self.buttons['Check']])
        button_box = widgets.HBox([left_box, right_box])

        self.m_pick = widgets.IntSlider(value=0, min=0, max=self.bouts_pd.index.size-1,step=1, 
                                        description="Bout candidate index")
        
        
        control_box = widgets.HBox([button_box,
                                  widgets.VBox([self.is_bout, self.is_confusing]),
                                    self.m_pick])
        
        link((self.m_pick, 'value'), (self.bout_counter, 'value'))

        self.update_bout()
        
        self.is_bout.observe(self.bout_checked, names='value')
        self.is_confusing.observe(self.confusing_checked, names='value')
        self.m_pick.observe(self.slider_change, names='value')
        
        all_containers = widgets.VBox([control_box, 
                                       self.figwidg_waveform, self.figwidg_spectrogram])
        display(all_containers)
#         display(button_box)
#         display(self.m_pick)
#         display(self.is_bout)
#         display(self.fig)
        
    def button_click(self, button):        
        self.bout_id = self.bout_counter.value
        curr_bout = self.bout_counter
        
        if button.description == 'Next':
            curr_bout.value += 1
        elif button.description == 'Prev':
            curr_bout.value -= 1
        elif button.description == 'Check':
            self.bouts_pd.loc[self.bout_id, 'bout_check'] = True
            self.bouts_pd.loc[self.bout_id, 'confusing'] = False
            curr_bout.value += 1
        elif button.description == 'Uncheck':
            self.bouts_pd.loc[self.bout_id, 'bout_check'] = False
            self.bouts_pd.loc[self.bout_id, 'confusing'] = False
            curr_bout.value += 1
        
        # handle the edges of the counter
        if curr_bout.value > self.m_pick.max:
            curr_bout.value = 0
            
        if curr_bout.value < self.m_pick.min:
            curr_bout.value = self.m_pick.max
    
    def slider_change(self, change):
        #logger.info('slider changed')
        #self.bout_counter = change.new
        #clear_output(True)
        self.update_bout()
        self.show()
            
    def bout_checked(self, bc):
#         print "bout checked"
#         print bc['new']
#         print self.motiff
        self.bouts_pd.loc[self.bout_id, 'bout_check'] = bc['new']
    
    def confusing_checked(self, bc):
#         print "bout checked"
#         print bc['new']
#         print self.motiff
        self.bouts_pd.loc[self.bout_id, 'confusing'] = bc['new']
    
    def update_bout(self):
        self.bout_id = self.bout_counter.value
        self.bout_series = self.bouts_pd.iloc[self.bout_id]
        
        self.is_bout.value = bool(self.bout_series['bout_check'])
        self.is_confusing.value = bool(self.bout_series['confusing'])
        
        self.x = self.bout_series['waveform'][::self.sub_sample]
        self.sxx = self.bout_series['spectrogram'][::self.sub_sample]
        
    def show(self):
        #self.fig.clf()
        #self.init_fig()
        # update
#         self.update_bout()
        #plot
        #logger.info('showing')
        
        # Show the figures
        with self.figwidg_waveform.batch_update():
            self.figwidg_waveform.data[0].y = self.x
            self.figwidg_waveform.data[0].x = np.arange(self.x.size) * self.sub_sample / self.s_f 
            
        with self.figwidg_spectrogram.batch_update():
            self.figwidg_spectrogram.data[0].z = np.sqrt(self.sxx[::-1])
            
        

viz_bout = VizBout(hparams, bout_pd)

In [24]:
np.where(viz_bout.bouts_pd['bout_check']==True)[0].size

34

### save it

In [33]:
hparams

{'num_freq': 1024,
 'preemphasis': 0.97,
 'frame_shift_ms': 5,
 'frame_length_ms': 10,
 'min_level_db': -55,
 'ref_level_db': 110,
 'sample_rate': 20000,
 'mel_filter': False,
 'num_mels': 1024,
 'fmin': 500,
 'fmax': 12000,
 'max_iters': 200,
 'griffin_lim_iters': 20,
 'power': 1.5,
 'read_wav_fun': <function __main__.read_wav_chan(wav_path: str, chan_id: int = 0) -> tuple>,
 'file_order_fun': <function __main__.sess_file_id(f_path)>,
 'min_segment': 30,
 'min_silence': 2000,
 'min_bout': 5000,
 'peak_thresh_rms': 0.55,
 'thresh_rms': 0.25,
 'mean_syl_rms_thresh': 0.3,
 'max_bout': 120000,
 'l_p_r_thresh': 100,
 'waveform_edges': 1000,
 'bout_auto_file': 'bout_auto.pickle',
 'bout_curated_file': 'bout_checked.pickle'}

In [25]:
### get the curated file path
##save to the curated file path
viz_bout.bouts_pd.to_pickle(sess_bouts_curated_file)
logger.info('saved curated bout pandas to pickle {}'.format(sess_bouts_curated_file))

2021-08-28 00:18:56,404 root         INFO     saved curated bout pandas to pickle /mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-07-18/sglx/bouts_ceciestunepipe/bout_curated.pickle


In [35]:
viz_bout.bouts_pd['file'][0]

'/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-14/sglx/0712_g0/wav_mic.wav'

In [36]:
viz_bout.bouts_pd.head(5)

,start_ms,end_ms,start_sample,end_sample,p_step,rms_p,peak_p,bout_check,file,len_ms,syl_in,n_syl,peaks_p,n_peaks,l_p_ratio,waveform,valid_waveform,valid,spectrogram,confusing
0,6340,12610,126800,252200,"[279.1424913127069, 237.28552677964962, 154.69...",32.49787,318.396805,False,/mnt/sphere/speech_bci/derived_data/s_b1253_21...,6270,"[[6340, 6385], [6560, 6605], [6695, 6740], [67...",14,"[13, 20, 35, 45, 59, 72, 85, 103, 121, 133, 14...",50,125.400000,"[-1847, -1860, -1872, -1873, -1852, -1881, -19...",True,True,"[[0.18799011171509866, 0.0, 0.0, 0.0, 0.0, 0.0...",False
1,15795,52345,315900,1046900,"[8.715509956604388, 15.467898681663929, 16.472...",32.49787,579.375792,True,/mnt/sphere/speech_bci/derived_data/s_b1253_21...,36550,"[[15795, 15870], [16290, 16340], [16415, 16685...",116,"[9, 100, 131, 142, 150, 157, 164, 175, 202, 21...",556,65.737410,"[906, 885, 877, 872, 864, 848, 856, 834, 859, ...",True,True,"[[0.05077423732344499, 0.0, 0.0, 0.0, 0.0, 0.0...",False
2,64885,94510,1297700,1890200,"[21.00765130413882, 49.58913937705543, 62.7825...",32.49787,569.916908,True,/mnt/sphere/speech_bci/derived_data/s_b1253_21...,29625,"[[64885, 65000], [65665, 66735], [67660, 67740...",93,"[6, 14, 169, 176, 187, 196, 205, 212, 219, 228...",475,62.368421,"[-2531, -2499, -2505, -2555, -2489, -2501, -24...",True,True,"[[0.23090914678582755, 0.0, 0.0, 0.0, 0.0, 0.0...",False
3,111180,146365,2223600,2927300,"[14.034488055830133, 13.201362167647732, 19.38...",32.49787,659.472493,True,/mnt/sphere/speech_bci/derived_data/s_b1253_21...,35185,"[[111180, 111500], [112865, 113230], [114070, ...",109,"[3, 12, 26, 38, 47, 55, 346, 357, 364, 371, 37...",544,64.678309,"[665, 665, 674, 704, 703, 701, 698, 732, 703, ...",True,True,"[[0.043878495887350835, 0.0, 0.0, 0.0, 0.0, 0....",False
4,190090,216300,3801800,4326000,"[11.342407418537071, 21.848205777005244, 29.74...",32.49787,550.972064,True,/mnt/sphere/speech_bci/derived_data/s_b1253_21...,26210,"[[190090, 190455], [191630, 192265], [192275, ...",92,"[9, 20, 33, 46, 53, 63, 312, 330, 339, 346, 35...",379,69.155673,"[-400, -392, -419, -429, -404, -386, -367, -36...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",False
